## RIVM Vragen & antwoorden nieuw coronavirus (COVID-19)

In verband met het coronavirus en de vele vragen betreffende dit onderwerp heb ik een dataset gemaakt met veelgestelde vragen en antwoorden van het RIVM. De vragen zijn afkomstig van deze website: https://www.rivm.nl/coronavirus/covid-19/vragen-antwoorden

Op deze website kan je redelijk snel voor een vraag die je hebt betreffende het coronavirus een antwoord vinden. Maar, met een chatbot zou die nog een stuk sneller kunnen gaan. Je stelt simpelweg je vraag in en je krijgt een bijbehorend antwoord. 

In de onderstaande code laad ik eerst de dataset waarin de antwoorden onder "Answer" staan en de context van de antwoorden onder "Context" staan. De interpretatie van deze kolommen dient als volgt geïnterpreteerd te worden:
- `Answer` is een mogelijk antwoord op een vraag.
- `Context` is een mogelijke context waarin het antwoord van toepassing zou kunnen zijn.

In [34]:
import numpy as np
import pandas as pd

# dataset coronavirus RIVM
pd.set_option('max_colwidth', 100)  # Increase column width
data = pd.read_excel("Corona_FAQ.xlsx", encoding='utf8')
data.head()

,Context,Answer
0,Welke maatregelen gelden in Nederland tegen verspreiding van het coronavirus?,"Iedereen in Nederland: blijf thuis bij klachten van neusverkoudheid of hoesten, keelpijn of koor..."
1,Welke maatregelen gelden in Nederland tegen verspreiding van het coronavirus?,Iedereen in heel Nederland: probeer zoveel mogelijk thuis te werken of de werktijden te spreiden.
2,Welke maatregelen gelden in Nederland tegen verspreiding van het coronavirus?,Voor kwetsbare personen (ouderen en personen met verminderde weerstand) geldt: vermijd grote gez...
3,Welke maatregelen gelden in Nederland tegen verspreiding van het coronavirus?,"Iedereen in Nederland wordt gevraagd om waar mogelijk 1,5 meter afstand van elkaar te bewaren. O..."
4,Welke maatregelen gelden in Nederland tegen verspreiding van het coronavirus?,Voor zorgpersoneel en personeel in vitale processen: er wordt veel van je gevraagd. Blijf pas th...


Om deze antwoorden te gebruiken voor een chatbot, gaan we eerst de bijbehorende answer encodings berekenen.

### Universal Sentence Encoder Multilingual

Nu kunnen we gebruik maken van USE om sentence encodings te maken voor de mogelijke antwoorden met context en de vragen! 

Hiervoor laden we de module, deze is hier te vinden: https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3

In [40]:
##### Use USE pretrained model to extract response encodings.
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import re

def preprocess_sentences(input_sentences):
    return [re.sub(r'(covid-19|covid)', 'coronavirus', input_sentence, flags=re.I) 
            for input_sentence in input_sentences]
        
# Load module containing USE
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

# Create response embeddings
response_encodings = module.signatures['response_encoder'](
        input=tf.constant(preprocess_sentences(data.Answer)),
        context=tf.constant(preprocess_sentences(data.Context)))['outputs']

Nu kunnen we gaan testen!

De sentence encoding voor de vraag wordt gemaakt in de variabele `question_encodings`. Op basis hiervan kiezen we het meest representatieve antwoord. 

Zoals beschreven in de blog is het meest representatieve antwoord diegene wiens embedding de hoek minimalizeert met de vraag embedding. Dit is equivalent met het bepalen van het maximum van de cosinus van de hoek, maar omdat elke embedding al een vector van lengte 1 is, hoeven we slechts het maximum van het inwendig product te bepalen. In de code hieronder is dat de regel:

``np.argmax(np.inner(question_encodings, response_encodings), axis=1)``

In [41]:
test_questions = [
    "Hoe zit het met zwangere vrouwen?",
    "Wat is de incubatietijd?",
    "Zijn huisdieren ook kwetsbaar voor het COVID-19?",
    "Zijn er medicijnen tegen het coronavirus?",
    "Zijn kinderen een risicogroep voor het coronavirus?",
    "Should I stay inside the house?",  # English questions are also possible.
    "Kann ich mit meinem Hund spazieren gehen?"  # As well as German, and all the other languages supported by use-multilingual.
]

# Create encodings for test questions
question_encodings = module.signatures['question_encoder'](
    tf.constant(preprocess_sentences(test_questions))
)['outputs']

# Get the responses
test_responses = data.Answer[np.argmax(np.inner(question_encodings, response_encodings), axis=1)]

# Show them in a dataframe
pd.DataFrame({'Test Questions': test_questions, 'Test Responses': test_responses})

,Test Questions,Test Responses
43,Hoe zit het met zwangere vrouwen?,Nee. We zien dat infectie met het virus hetzelfde verloop heeft als bij vrouwen die niet zwanger...
86,Wat is de incubatietijd?,Bij het nieuwe coronavirus varieert de incubatietijd van 2 tot 12 dagen. Voor de zekerheid gebru...
87,Zijn huisdieren ook kwetsbaar voor het COVID-19?,Er is op dit moment geen bewijs dat gezelschapsdieren ziek kunnen worden van het nieuwe coronavi...
57,Zijn er medicijnen tegen het coronavirus?,Er is (nog) geen medicijn tegen het nieuwe coronavirus. Paracetamol en ibuprofen helpen niet om ...
16,Zijn kinderen een risicogroep voor het coronavirus?,Op dit moment krijgen kinderen bijna nooit klachten van het nieuwe coronavirus. Kinderen zijn du...
103,Should I stay inside the house?,"Als u kucht, hoest en/of verkouden bent, gold al: blijf thuis. Krijgt u daar ook koorts bij, dan..."
45,Kann ich mit meinem Hund spazieren gehen?,"Als je geen klachten hebt zoals koorts, hoest of verkoudheidsklachten, kun je gewoon je hond uit..."


### Conclusie

Zoals je ziet, geeft de chatbot correcte antwoorden op vragen! De reden dat we COVID-19 moeten vervangen in de dataset met coronavirus komt doordat het model nog nooit eerder het woord COVID-19 gezien heeft (maar coronavirus wel). 

Voel je vrij om meerdere vragen te testen, dit kan je doen door een vraag toe te voegen aan `test_questions`.

Voor verdere vragen kan je natuurlijk ook [direct met ons contact opnemen](mailto:ai@wearetriple.com)!